# **Exploratory Data Analysis (EDA) & Preprocessing**
Este notebook contiene el análisis exploratorio de datos (EDA) y el preprocesamiento del dataset **Life Expectancy**.  
El objetivo es preparar los datos para ser usados en los distintos modelos de regresión.

En este notebook, se cargan los datos, se hace exploración inicial, tratamiento de nulos, outliers, codificación, escalado y exportación de datasets listos (X_scaled, X_no_scaled, target_y).

## **Paso 1. Cargar librerías**
Importamos las librerías de análisis y visualización que vamos a usar:

* pandas → manipulación de datos.
* matplotlib/seaborn → visualizaciones estáticas.
* plotly → gráficos interactivos.
* sklean → herramientas ML y DS.
* missingno → visualización de datos nulos
* StandardScaler from scikit learn.preprocessing → Escalado de variables

In [ ]:
# ===================================
# 1. Importación de librerías
# ===================================

import pandas as pd                                                             # Manipulación de datos y análisis
import numpy as np                                                              # Cálculos numéricos                               
import matplotlib.pyplot as plt                                                 # Gráficos básicos (histogramas, scatter, etc.).
import seaborn as sns                                                           # Gráficos estadísticos con menos código (heatmaps, distribuciones…)
import missingno as msno                                                        # Visualización de datos faltantes
from sklearn.preprocessing import StandardScaler, MinMaxScaler                  # Escalado de variables (normalización y estandarización)


# Configuración de estilo
sns.set_style("whitegrid")
plt.rcParams.update({"figure.figsize": (8,5), "axes.titlesize": 14, "axes.labelsize": 12})


## **Paso 2. Carga del dataset (../data/life_expectancy_data.csv)**

In [ ]:
# ===================================
# 2. Cargar dataset original
# (el archivo debe estar en la carpeta data/)
# ===================================

df = pd.read_csv("../data/life_expectancy_data.csv")

# Vista general
print("Dimensiones:", df.shape)
display(df.head(10))  # 👈 muestra primeras 10 filas en formato tabla




## **Paso 3. EDA inicial: Información básica**

En este bloque realizamos una **exploración general del dataset**:
- Revisamos las primeras filas para entender la estructura de los datos.
- Comprobamos el número de filas y columnas.
- Inspeccionamos el tipo de variables (numéricas y categóricas).
- Verificamos si existen valores nulos en las columnas.

Este paso es clave para familiarizarnos con el dataset y planificar los siguientes pasos de limpieza y preprocesamiento.

In [ ]:
# ===================================
# 3. EDA inicial: Información básica
# ===================================

# Revisar nombres de columnas
print("\nInformación del dataset:\n")
df.info()

# Separador visual
print("\n" + "="*50 + "\n")


# Limpiar espacios en nombres de columnas
df.columns = df.columns.str.strip()


# ===================================
# Valores nulos
# ===================================
print("\nValores nulos por columna (top 10):\n")
print(df.isnull().sum().sort_values(ascending=False).head(10))


# ===================================
# Visualización de nulos con missingno
# ===================================

# Mapa de calor de nulos
msno.matrix(df, figsize=(10, 5), color=(0.2, 0.4, 0.7))
plt.title("Mapa de calor de valores nulos")
plt.show()



# Definir variable objetivo
target_col = "Life expectancy"  

# Distribución de la variable objetivo
sns.histplot(df[target_col], kde=True, color="blue")
plt.title("Distribución de la variable objetivo (Life Expectancy)")
plt.show()

# Separador visual
print("\n" + "="*50 + "\n")





## **Paso 4. Correlaciones iniciales**

En este bloque analizamos las **relaciones lineales** entre las variables numéricas del dataset.  
1. Primero generamos un **mapa de calor** (*heatmap*) para visualizar todas las correlaciones entre pares de variables.  
   - Esto nos permite detectar posibles redundancias entre predictores (multicolinealidad).  
2. Después calculamos y mostramos las correlaciones de **cada variable con la variable objetivo** (`Life expectancy`).  
   - De esta forma identificamos qué variables están más asociadas con la esperanza de vida y cuáles aportan menos información.  
3. Finalmente graficamos estas correlaciones en un **gráfico de barras ordenado**, lo que facilita interpretar qué predictores tienen más peso potencial en el modelo.

In [ ]:
# ===================================
# 4. Correlaciones iniciales numéricas y correlaciones con la variable objetivo
# ===================================


# Seleccionar numéricas
df_num = df.select_dtypes(include=["int64", "float64"])

# --- Heatmap de correlaciones numéricas
plt.figure(figsize=(12, 8))
sns.heatmap(
    df_num.corr(),
    cmap="coolwarm",
    annot=True,         #  muestra los valores dentro de cada celda
    fmt=".2f",          #  formato con 2 decimales
    annot_kws={"size": 8}  #  tamaño del texto
)
plt.title("Mapa de calor de correlaciones numéricas", fontsize=14)
plt.show()

# --- Correlaciones con la variable objetivo
correlations = df_num.corr()[target_col].sort_values(ascending=False)

print("📊 Correlaciones con la variable objetivo:")
print(correlations)

# Visualización en gráfico de barras
plt.figure(figsize=(8, 10))
sns.barplot(
    x=correlations.values,
    y=correlations.index,
    hue=correlations.index,  # usar cada variable como hue
    palette="coolwarm",
    dodge=False,
    legend=False
)


plt.title(f"Correlación de variables numéricas con {target_col}", fontsize=14)
plt.xlabel("Coeficiente de correlación (Pearson)")
plt.ylabel("Variables")
plt.show()


## **Paso 5: Outliers (boxplots + IQR)**

Los **outliers** son valores que se alejan mucho del resto de observaciones y pueden distorsionar las métricas de entrenamiento.  
En este bloque aplicamos dos técnicas principales:

1. **Boxplots (diagramas de caja y bigotes)**:  
   - Permiten identificar valores extremos en la distribución de cada variable numérica.  
   - Los puntos fuera de los bigotes suelen considerarse outliers.

2. **Método del IQR (Interquartile Range)**:  
   - Calculamos los percentiles Q1 (25%) y Q3 (75%).  
   - Definimos un rango intercuartílico (IQR = Q3 - Q1).  
   - Todo valor < Q1 - 1.5×IQR o > Q3 + 1.5×IQR se considera un outlier.  

Este análisis nos ayuda a decidir si necesitamos **tratar o eliminar** los valores extremos antes de entrenar los modelos.
  
En general:
- Algoritmos basados en distancias o regresión lineal son sensibles a outliers.  
- Los árboles de decisión y ensembles (Random Forest, XGBoost) son más robustos a su presencia.

In [ ]:
# ===================================
# 5. Outliers con IQR
# ===================================

# --- Boxplots para variables numéricas
num_cols = df.select_dtypes(include=["float64","int64"]).columns

plt.figure(figsize=(15, 8))
sns.boxplot(data=df_num, orient="h", palette="Set2", showfliers=True)
plt.title("Boxplots de variables numéricas", fontsize=14)
plt.show()

for col in num_cols[:5]:  # mostramos solo primeras 5 para no saturar
    sns.boxplot(x=df[col], color="skyblue")
    plt.title(f"Boxplot de {col}")
    plt.show()

# Detectar outliers con método IQR
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    return series[(series < lower) | (series > upper)]

outliers_count = {}
for col in num_cols:
    outliers_count[col] = len(detect_outliers_iqr(df[col].dropna()))

print("Número de outliers detectados por variable:")
print(pd.Series(outliers_count).sort_values(ascending=False).head(10))


## **Paso 6: Preprocesamiento**

En este bloque transformamos el dataset para que pueda ser utilizado por distintos modelos de machine learning:

1. Eliminamos columnas con demasiados valores nulos (>40%-50%) y que aquellas que no aporten nada (ej. identificadores, códigos sin significado).
2. Rellenamos valores faltantes:  
   - Numéricas: con la mediana (robusta a outliers).  
   - Categóricas: con la categoría `"Unknown"`.  
3. Reducimos categorías poco frecuentes en variables categóricas (ej. países con pocas observaciones) agrupándolas como `"Other"`.  
4. Aplicamos **One-Hot Encoding** a las variables categóricas → convierte categorías en columnas binarias (0/1).  

---


💡 **Nota: Tratamiento de valores nulos en la variable objetivo (Life expectancy)**

Durante el preprocesamiento detectamos que la variable objetivo `Life expectancy` contenía **10 valores nulos**.

Esto es importante porque un modelo de Machine Learning **no puede entrenar** sin la variable objetivo (`y`).

Opciones posibles:

1. **Eliminar filas con NaN en `y` (recomendado)**  
   - Justificación: al ser solo 10 de 2938 filas (~0.3%), el impacto es mínimo.  
   - Evitamos introducir sesgo artificial.  

2. **Imputar `y` con media o mediana (no recomendado)**  
   - Se puede hacer, pero añade un valor inventado que no estaba en el dataset original.  
   - Puede distorsionar la distribución y las métricas del modelo.

 En nuestro caso aplicamos la **opción 1 (eliminación de filas con NaN en `y`)**, quedando finalmente **2928 registros válidos**.




In [ ]:
# ===================================
# 6. Preprocesamiento
# ===================================

# --- Separar target ---
y = df[target_col]
df_features = df.drop(columns=[target_col])

# --- Eliminar columnas con más del 40% de nulos ---
missing = df_features.isnull().mean()
cols_to_drop = missing[missing > 0.4].index
df_features = df_features.drop(columns=cols_to_drop)

# --- Rellenar nulos ---
for col in df_features.select_dtypes(include=["int64","float64"]).columns:
    df_features[col] = df_features[col].fillna(df_features[col].median())

for col in df_features.select_dtypes(include=["object"]).columns:
    df_features[col] = df_features[col].fillna("Unknown")

# --- Reducir categorías (si >20 únicas → agrupar en "Other") ---
for col in df_features.select_dtypes(include=["object"]).columns:
    if df_features[col].nunique() > 20:
        top = df_features[col].value_counts().index[:20]
        df_features[col] = df_features[col].apply(lambda x: x if x in top else "Other")

# --- One-hot encoding ---
df_features = pd.get_dummies(df_features, drop_first=True)


# ===================================
# Tratamiento de nulos en y
# ===================================

#  --- LIMPIEZA adicional de la variable objetivo ---
print(f"Valores nulos antes en la variable objetivo y ({target_col}):", y.isna().sum())

# Eliminar filas con y nulo
mask = y.notna()
df_features = df_features.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)

print("Valores nulos en y (después):", y.isna().sum())
print("Dimensiones finales después de limpiar target:", df_features.shape, y.shape)




## **Paso 7: Escalado y exportación**

1. Creamos dos versiones del dataset:  
   - `X_no_scaling`: para modelos robustos a la escala (árboles, Random Forest, XGBoost).  
   - `X_scaled`: aplicamos `StandardScaler`, necesario en modelos sensibles a magnitud (Regresión Lineal, KNN, SVM).
   - `target_y`: dataframe con la variable objetivo `Life expectancy`

In [ ]:
# ===================================
# 7. Escalado y exportación
# ===================================
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(df_features), columns=df_features.columns)

# --- Comprobación final de nulos ---
print("🔎 Comprobación final de nulos en datasets procesados:")
print(" - X_no_scaling:", df_features.isna().sum().sum())
print(" - X_scaled:", X_scaled.isna().sum().sum())
print(" - y:", y.isna().sum())

# Guardar datasets
df_features.to_csv("../data/processed/features_no_scaling.csv", index=False)
X_scaled.to_csv("../data/processed/features_scaled.csv", index=False)
y.to_csv("../data/processed/target_y.csv", index=False)

print("✅ Datasets procesados guardados en data/processed/")
